# Testing Custom Timeouts


## Setup Seldon Core

Use the setup notebook to [Setup Cluster](https://docs.seldon.io/projects/seldon-core/en/latest/examples/seldon_core_setup.html#Setup-Cluster) with [Ambassador Ingress](https://docs.seldon.io/projects/seldon-core/en/latest/examples/seldon_core_setup.html#Ambassador) and [Install Seldon Core](https://docs.seldon.io/projects/seldon-core/en/latest/examples/seldon_core_setup.html#Install-Seldon-Core). Instructions [also online](https://docs.seldon.io/projects/seldon-core/en/latest/examples/seldon_core_setup.html).

Illustration of both REST and gRPC requests. 

### Short timeouts deployment file
Below is the outputs of the Seldon config file required to set custom timeouts.

First we'll show how we can set a short lived example by setting the `"seldon.io/rest-read-timeout":"1"` annotation.

In [ ]:
%%writefile resources/model_short_timeouts.json
{
    "apiVersion": "machinelearning.seldon.io/v1alpha2",
    "kind": "SeldonDeployment",
    "metadata": {
        "labels": {
            "app": "seldon"
        },
        "name": "seldon-deployment-example"
    },
    "spec": {
        "annotations": {
            "project_name": "FX Market Prediction",
            "deployment_version": "v1",
	    "seldon.io/rest-read-timeout":"1"
        },
        "name": "test-deployment",
        "predictors": [
            {
                "componentSpecs": [{
                    "spec": {
                        "containers": [
                            {
                                "image": "seldonio/mock_classifier:1.0",
                                "imagePullPolicy": "IfNotPresent",
                                "name": "classifier",
                                "resources": {
                                    "requests": {
                                        "memory": "1Mi"
                                    }
                                }
                            }
                        ],
                        "terminationGracePeriodSeconds": 20
                    }
                }],
                "graph": {
                    "children": [],
                    "name": "classifier",
                    "endpoint": {
			"type" : "REST"
		    },
                    "type": "MODEL"
                },
                "name": "fx-market-predictor",
                "replicas": 1,
		"annotations": {
		    "predictor_version" : "v1"
		}
            }
        ]
    }
}

## Seldon Deployment with Short Timeout

We can then apply this Seldon Deployment file we defined above in the `seldon` namespace.

In [ ]:
!kubectl apply -f resources/model_short_timeouts.json -n seldon

And wait until the resource runs correctly.

In [ ]:
!kubectl rollout status deploy/$(kubectl get deploy -l seldon-deployment-id=seldon-deployment-example -o jsonpath='{.items[0].metadata.name}')

Now that it's deployed we can send requests to the model using the python client

In [ ]:
from seldon_core.seldon_client import SeldonClient
sc = SeldonClient(deployment_name="seldon-deployment-example",namespace="seldon")

This next request should fail as the timeout is too short

In [ ]:
r = sc.predict(gateway="ambassador",transport="rest")
print(r)

Delete this graph and recreate one with a longer timeout

In [ ]:
!kubectl delete -f resources/model_short_timeouts.json

## Seldon Deployment with Long Timeout

Now we can have a look at how we can set a deployment with a longer timeout by setting the annotations `"seldon.io/rest-timeout":"100000"`, and `"seldon.io/grpc-timeout":"100000"`.

In [ ]:
%%writefile resources/model_long_timeouts.json
{
    "apiVersion": "machinelearning.seldon.io/v1alpha2",
    "kind": "SeldonDeployment",
    "metadata": {
        "labels": {
            "app": "seldon"
        },
        "name": "model-long-timeout"
    },
    "spec": {
        "annotations": {
            "deployment_version": "v1",
	    "seldon.io/rest-timeout":"100000",
	    "seldon.io/grpc-timeout":"100000"
        },
        "name": "long-to",
        "predictors": [
            {
                "componentSpecs": [{
                    "spec": {
                        "containers": [
                            {
                                "image": "seldonio/mock_classifier_grpc:1.5",
                                "imagePullPolicy": "IfNotPresent",
                                "name": "classifier",
                                "resources": {
                                    "requests": {
                                        "memory": "1Mi"
                                    }
                                }
                            }
                        ],
                        "terminationGracePeriodSeconds": 20
                    }
                }],
                "graph": {
                    "children": [],
                    "name": "classifier",
                    "endpoint": {
			"type" : "GRPC"
		    },
                    "type": "MODEL"
                },
                "name": "test",
                "replicas": 1,
		"annotations": {
		    "predictor_version" : "v1"
		}
            }
        ]
    }
}

And we can apply it to deploy the model with long timeouts.

In [ ]:
!kubectl apply -f resources/model_long_timeouts.json -n seldon

In [ ]:
!kubectl rollout status deploy/$(kubectl get deploy -l seldon-deployment-id=model-long-timeout -o jsonpath='{.items[0].metadata.name}')

This next request will work given that the timeout is much longer.

In [ ]:
r = sc.predict(gateway="ambassador",transport="grpc",deployment_name="model-long-timeout")
print(r)